In [39]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

In [40]:
# import files
full_df = pd.read_csv('../sample_query/output_all.csv')

# state metric in focus
metric = 'Hateful Score'

In [41]:
# Function to attempt conversion to float
def convert_to_float(col):
    try:
        return pd.to_numeric(col, errors='raise')
    except ValueError:
        return col

# Apply conversion function to each column
full_df = full_df.apply(convert_to_float)

In [42]:
full_df['age'] = pd.to_numeric(full_df['Hateful Score'])

In [43]:
# processing time statistics (capped at 60s)
full_df['wait_elapsed'].describe()

count               2,974.00
mean                    9.98
std                     2.89
min                     5.00
25%                     7.46
50%                    10.03
75%                    12.49
max                    15.00
Name: wait_elapsed, dtype: float64

In [44]:

agg_values = full_df.groupby('annotation_selected')[metric].describe()
agg_values

,count,mean,std,min,25%,50%,75%,max
annotation_selected,,,,,,,,
hateful,"2,300.00",0.95,0.33,-1.00,1.00,1.00,1.00,1.00
no_match,38.00,-1.00,0.00,-1.00,-1.00,-1.00,-1.00,-1.00
non-hateful,618.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00
undecided,18.00,-1.00,0.00,-1.00,-1.00,-1.00,-1.00,-1.00


In [45]:
# get group by annotation values and get statistics
column_list = ['t_function', 't_direction', 'p_target']

def percentile(n):
    def percentile_(x):
        return x.quantile(n)
    percentile_.__name__ = 'percentile_{:02.0f}'.format(n*100)
    return percentile_


# if metric out is an float
for column in column_list:
    print(f'Overall for {column}')
    quartile_list = [0, 0.25, 0.5, 0.75, 1]
    quartile_list_str = ['{:.0f}%'.format(n*100) for n in quartile_list]
    agg_values = full_df.groupby([column, 'annotation_selected'])[metric].describe(percentiles=quartile_list)#.sort_values('mean', ascending=False)
    agg_values = agg_values[agg_values['count']>10]
    print(agg_values)
    agg_values.to_csv(f'../sample_query/{column}.csv')

    output_txt = ''
    # calculate quartiles and cycle through each row
    output_txt += f'Quartiles with examples for {column} \n'
    for row in agg_values.iterrows():
        output_txt+=(f'{column}: {row[0]} \n')
        for quartile_str in quartile_list_str:
            quartile_value = row[1][quartile_str]
            output_txt += f'Quartile {quartile_str} = {quartile_value} \n'
            series = full_df[
                (full_df[column] == row[0][0]) & # match column value
                (full_df['annotation_selected']==row[0][1]) & # match ground truth value
                (full_df[metric] == quartile_value)
            ]['c_testcase_x']
            output_txt+=f'{series.sample(n=min(3, len(series)))} \n'
        with open(f'../sample_query/{column}.txt', 'w') as file:
            file.write(output_txt)


Overall for t_function
                                                      count                 mean                  std                  min                   0%                  25%                  50%                  75%                 100%                  max
t_function         annotation_selected                                                                                                                                                                                                                  
counter_quote_nh   non-hateful                       113.00                 1.00                 0.00                 1.00                 1.00                 1.00                 1.00                 1.00                 1.00                 1.00
counter_ref_nh     non-hateful                        95.00                 1.00                 0.00                 1.00                 1.00                 1.00                 1.00                 1.00                 1.00   